#### Project 3: The worm algorithm for the 6-vertex model (Monte Carlo)

Implement the worm algorithm for the 6-vertex (ice-type) model (https://en.wikipedia.org/wiki/Ice-type_model) at $T = \infty$ (where all allowed configurations are equally likely).

Calculate the exponent $a$ of the correlation function $\langle s_0 s_r \rangle \propto \frac{1}{r^a}$.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
class six_vertex_model(object):
    """Ice-type (6-vertex) model class with periodic boundary conditions.
    
    Attributes
    ----------
    Lx : int
        Lattice X size.
    Ly : int
        Lattice Y size.
    N : int
        Number of vertices.
    lattice : list
        Horizontal (hor) & vertical (ver) sublattices.
        Shape: ((Ly, Lx), (Lx, Ly)).
    
    Methods
    ----------
    - plot()
    - energy(energies)
    - get_vertex_in_out(x, y)
    - get_vertex_charge(x, y)
    - get_next_vertex(x, y, flip)
    - flip_spin(x, y, flip)
    - flip_next(x, y, flip)
    - check_valid_config()
    - propagate_2_defects()
    - thermalize(N)
    - measure_correlations()
    """
    
    def __init__(self, Lx, Ly, hor=[], ver=[]): 
        '''hor, ver: initial lattice configuration.
        '''
        self.Lx = Lx
        self.Ly = Ly
        self.N  = Lx * Ly;
        
        # periodic BC: hor[_][0] = hor[_][Lx+1] & ver[_][0] = ver[_][Ly+1]
        if hor == [] or ver == []:
            hor = np.ones((Ly, Lx), dtype=np.int8) 
            ver = np.ones((Lx, Ly), dtype=np.int8)
            # initial state: alternate directions from line to line
            hor[1::2] *= -1
            ver[1::2] *= -1
            self.lattice = np.array([hor, ver]) # shape: ((Ly, Lx), (Lx, Ly))
        else:
            assert hor.shape == (Ly, Lx) and ver.shape == (Lx, Ly)
            self.lattice = np.array([hor, ver]) # shape: ((Ly, Lx), (Lx, Ly))
            self.check_valid_config()
        
        # self.n = np.array([?]*6) # (dummy) how many in each of the 6 config types
    
    def plot(self):
        '''Plot hor & ver sublattices (´ver´ transposed for graphical purposes).
        '''
        print('Horizontal spins:')
        print(self.lattice[0], '\n')
        print('Vertical spins:')
        print(self.lattice[1].transpose(), '\n')
    
    def energy(self, energies=[1.]*6):
        '''(dummy) energies : list with energies of the 6 configurations.
        '''
        return self.N # T->Inf: all equally likely
        # return np.sum(self.n * np.array(energies))

    def get_vertex_in_out(self, x, y):
        '''Return spin values: np.array([spin_left, spin_right, spin_up, spin_down])
        relative to vertex (x, y).
        In  :  1
        Out : -1
        '''
        spin_left  = self.lattice[0, x, y]
        spin_right = self.lattice[0, x, (y+1) % self.Lx] * -1
        spin_up    = self.lattice[1, y, x] * -1
        spin_down  = self.lattice[1, y, (x+1) % self.Ly]
        return np.array([spin_left, spin_right, spin_up, spin_down])
    
    def get_vertex_charge(self, x, y):
        '''Return magnetic charge of vertex (x, y).
        '''
        return np.sum(self.get_vertex_in_out(x, y)) / 2

    def get_next_vertex(self, x, y, flip):
        '''Return coord of neighbour vertex in the flip direction & flip
        relative to that vertex.
        x, y : 1st vertex coord
        flip : 0 -> left, 1 -> right, 2 -> up, 3 -> down
        '''
        if flip == 0:   # left
            return x, (y-1) % self.Lx, 1
        elif flip == 1: # right
            return x, (y+1) % self.Lx, 0
        elif flip == 2: # up
            return (x-1) % self.Ly, y, 3
        else:           # down
            return (x+1) % self.Ly, y, 2
    
    def flip_spin(self, x, y, flip):
        '''Create 2 defects by flipping 1 spin.
        x, y : vertex coord
        flip : 0 -> left, 1 -> right, 2 -> up, 3 -> down
        '''
        if flip == 0:   # left
            self.lattice[0, x, y] *= -1
        elif flip == 1: # right
            self.lattice[0, x, (y+1) % self.Lx] *= -1
        elif flip == 2: # up
            self.lattice[1, y, x] *= -1
        else:           # down
            self.lattice[1, y, (x+1) % self.Ly] *= -1
    
    def flip_next(self, x, y, flipped):
        '''Propagate defect by flipping randomly one of the vertex's other spins,
        subject to the ice rule (2-in, 2-out).
        Return next vertex coord & new flip relative to that vertex.
        x, y    : vertex coord
        flipped : 0 -> left, 1 -> right, 2 -> up, 3 -> down
        '''
        in_or_out = self.get_vertex_in_out(x, y)
        flipped_dir = in_or_out[flipped] # in: 1, out: -1
        
        in_or_out[flipped] = 0 # remove old direction from possibilities
        # possible_mask = (in_or_out == flipped_dir) # boolean mask
        possible = np.arange(4)[in_or_out == flipped_dir] # 2 new flip directions
        # print('2 possible flip directions:', possible)
        
        flip_new = -1
        prob = np.random.random() # 50/50 chance
        if prob < 0.5:
            flip_new = possible[0]
        else:
            flip_new = possible[1]
        # print('Chosen direction:', flip_new)
        
        self.flip_spin(x, y, flip_new)
        return self.get_next_vertex(x, y, flip_new)
    
    def check_valid_config(self):
        '''Check if system respects the ice rule (2-in, 2-out).
        Return number of defects (charged vertices).
        '''
        charged = 0
        for x in range(self.Ly):
            for y in range(self.Lx):
                if self.get_vertex_charge(x, y) != 0:
                    charged += 1
        
        if charged == 0:
            print('6-vertex configuration is valid. No finite-charge vertices.')
        else:
            print('6-vertex configuration is not valid!', charged, 'finite-charge vertices.')
    
    def propagate_2_defects(self):
        '''Flip 1 spin, create 2 defective vertices & propagate one randomly until
        they meet & annihilate each other.
        '''
        x = np.random.randint(self.Lx) # vertex x coord
        y = np.random.randint(self.Ly) # vertex y coord
        flip = np.random.randint(4) # 0: left, 1: right, 2: up, 3: down
        # print('Vertex:    ', x, y, '\tSpin-flip:', flip)
        
        self.flip_spin(x, y, flip)
        xf, yf, _ = self.get_next_vertex(x, y, flip) # end vertex coord (x_f, y_f)
        
        propagate = True
        while propagate:
            x, y, flip = self.flip_next(x, y, flip)
            # print('\nNew vertex:', x, y, '\tNew flip:', flip)
            if (x, y) == (xf, yf): # defects met & annihilated
                propagate = False
    
    def thermalize(self, N=1000):
        '''Thermalize system by running ´propagate_2_defects´ N times.
        '''
        for _ in range(N):
            self.propagate_2_defects()
    
    def measure_correlations(self, corr='all'):
        '''Measure spin correlations with distance on square lattice.
        Errors decreased using translational & rotational invariance.
                
        Return (corr == 'all'):
            1) corrhor_hor : horizontal correlations of horizontal spins. Size: L - 1
            2) corrver_ver :  vertical  correlations of  vertical  spins. Size: L - 1
            3) corrver_hor :  vertical  correlations of horizontal spins. Size: L - 1
            4) corrhor_ver : horizontal correlations of  vertical  spins. Size: L - 1
            5) corr_diag1  :  diagonal  correlations.                 Size: 2 * L - 1
            6) corr_diag2  : antidiagonal correlations.               Size: 2 * L - 1
            
        Due to rotational invariance:
        - 1) & 2) should be the same.
        - 3) & 4) should be the same.
        - 5) & 6) should be the same.
        
        Return (corr == 'rot_invar'):
            - corr_along : correlations along spin direction.            Size: L - 1
            - corr_perp  : correlations perpendicular to spin direction. Size: L - 1
            - corr_diag  : diagonal correlations.                    Size: 2 * L - 1
        '''
        assert self.Lx == self.Ly # square lattice
        L = self.Lx
        
        corrhor_hor = np.empty((L, L - 1))
        corrver_ver = np.empty((L, L - 1))
        corrver_hor = np.empty((L, L - 1))
        corrhor_ver = np.empty((L, L - 1))
        corr_diag1  = np.zeros((L, 2*L - 1))
        corr_diag2  = np.zeros((L, 2*L - 1))

        for i in range(L):
            # print(self.lattice[0, :, i, np.newaxis], self.lattice[0, :, np.r_[i+1:L, :i]])
            corrhor_hor[i] = np.sum(self.lattice[0, :, i, np.newaxis] *
                                    self.lattice[0, :, np.r_[i+1:L, :i]].transpose(), 0)
            corrver_ver[i] = np.sum(self.lattice[1, :, i, np.newaxis] *
                                    self.lattice[1, :, np.r_[i+1:L, :i]].transpose(), 0)

            corrver_hor[i] = np.sum(self.lattice[0, i] *
                                    self.lattice[0, np.r_[i+1:L, :i]], 1)
            corrhor_ver[i] = np.sum(self.lattice[1, i] *
                                    self.lattice[1, np.r_[i+1:L, :i]], 1)
            # print(corrhor_hor[i], corrver_ver[i])
            # print(corrhor_hor[i], corrver_ver[i])
            
            # diagonal correlations
            for k in range(L):
                # diagonals
                spin_origin, _, _, spin_origin2 = self.get_vertex_in_out((i+k) % L, k)
                corr_diag1[i, 0] += spin_origin * spin_origin2
                corr_diag1[i,-1] += spin_origin2 * spin_origin
                
                # antidiagonals
                spin_antiorigin, _, spin_antiorigin2, _ = self.get_vertex_in_out((i-k) % L, k)
                corr_diag2[i, 0] += spin_antiorigin * -spin_antiorigin2
                corr_diag2[i,-1] += -spin_antiorigin2 * spin_antiorigin

                for j in range(1, L):
                    spin_left, _, _, spin_down = self.get_vertex_in_out((i+k+j) % L, (k+j) % L)
                    corr_diag1[i, 2*j-1] += spin_origin * spin_left
                    corr_diag1[i, 2*j]   += spin_origin * spin_down
                    corr_diag1[i, 2*j-2] += spin_origin2 * spin_left
                    corr_diag1[i, 2*j-1] += spin_origin2 * spin_down
            
                    spin_left, _, spin_up, _ = self.get_vertex_in_out((i-k-j) % L, (k+j) % L) 
                    corr_diag2[i, 2*j-1] += spin_antiorigin * spin_left
                    corr_diag2[i, 2*j]   += spin_antiorigin * -spin_up
                    corr_diag2[i, 2*j-2] += -spin_antiorigin2 * spin_left
                    corr_diag2[i, 2*j-1] += -spin_antiorigin2 * -spin_up
            # print(corr_diag1[i], corr_diag2[i])
            
#             # Foda-se. Fo da se ---> implemented here without get_vertex_in_out(x, y)...
#             for k in range(L):
#                 for j in range(L):
#                     # print(2*j, 1,     (i+j) % L, (k+i+j+1) % L)
#                     # print(2*j, 0, (k+i+j+1) % L,   (i+j+1) % L)
#                     corr_diag1[k, 2*j] += self.lattice[0, (k+i) % L, i] * \
#                                           self.lattice[1, (i+j) % L, (k+i+j+1) % L]
#                     corr_diag1[k, 2*j] += self.lattice[1, i, (k+i+1) % L] * \
#                                           self.lattice[0, (k+i+j+1) % L, (i+j+1) % L]
#                     if j == L-1:
#                         break # cycle in j
#                     # print(2*j+1, 0, (k+i+j+1) % L,   (i+j+1) % L)
#                     # print(2*j+1, 1,   (i+j+1) % L, (k+i+j+2) % L)
#                     corr_diag1[k, 2*j+1] += self.lattice[0, (k+i) % L, i] * \
#                                             self.lattice[0, (k+i+j+1) % L, (i+j+1) % L]
#                     corr_diag1[k, 2*j+1] += self.lattice[1, i, (k+i+1) % L] * \
#                                             self.lattice[1, (i+j+1) % L, (k+i+j+2) % L]
#             print(i, corr_diag1, '\n')
        
        if corr == 'all': # separate for initial comparison
            corrhor_hor = np.sum(corrhor_hor, 0) / (L * L)
            corrver_ver = np.sum(corrver_ver, 0) / (L * L)
            corrver_hor = np.sum(corrver_hor, 0) / (L * L)
            corrhor_ver = np.sum(corrhor_ver, 0) / (L * L)
            corr_diag1  = np.sum(corr_diag1, 0)  / (L * 2*L)
            corr_diag2  = np.sum(corr_diag2, 0)  / (L * 2*L)
        
            return corrhor_hor, corrver_ver, corrver_hor, corrhor_ver, corr_diag1, corr_diag2
        
        elif corr == 'rot_invar': # average due to invariance
            corr_along = np.sum(corrhor_hor + corrver_ver, 0) / (2 * L * L)
            corr_perp  = np.sum(corrver_hor + corrhor_ver, 0) / (2 * L * L)
            corr_diag  = np.sum(corr_diag1 + corr_diag2, 0)  / (2 * L * 2*L)
            
            return corr_along, corr_perp, corr_diag

In [5]:
# Testing:
L = 4
icetype = six_vertex_model(L, L)
# print('Energy:', icetype.energy([1, 1, 1, 1, 1, 1]))
# print('Vertex (0, 0):', icetype.get_vertex_in_out(0, 0))
# print('Other defect:', icetype.get_other_defect(2, 2, 1), '\n')
# icetype.plot()

# x0, y0, flip0 = 2, 2, 1
# icetype.flip_spin(x0, y0, flip0)
# icetype.plot()
# icetype.flip_next(x0, y0, flip0)
# icetype.plot()
# icetype.check_valid_config()

# icetype.propagate_2_defects()
# icetype.thermalize(10000)
# icetype.plot()

# print('corrhor_hor, corrver_ver, corrver_hor, corrhor_ver, corr_diag1, corr_diag2')
# print('corr_along, corr_perp, corr_diag')
# icetype.measure_correlations('all'), icetype.measure_correlations('rot_invar')

In [6]:
def run_worm_simulation(Lx, Ly, Niter, corr_type='all'):
    '''Run worm algorithm on 6-vertex model.
    Return (corr_type == 'all'):       correlations list (shape: (Niter, 6))
    Return (corr_type == 'rot_invar'): correlations list (shape: (Niter, 3))
    '''
    icetype = six_vertex_model(Lx, Ly)
    corrs = []
    
    icetype.thermalize(Niter//10)
    for i in range(Niter):
        icetype.propagate_2_defects()
        corrs.append(icetype.measure_correlations(corr_type))
    return corrs

In [7]:
Ls = [10] # [10, 20, 30, 40, 50]
Ni = 100000 / Ls[0]**2
Nbins = 20
Niter = Ni * Nbins
corrs = [] # shape: (Ls, Niter, 3/6 corr types)
corrsavg = [] # shape: (Ls, Nbins, 3/6 corr types)

# run worm algorithm & get correlations
for i, L in enumerate(Ls):
    corrs.append(run_worm_simulation(L, L, Niter, corr_type='all'))
    
    # Binning analysis
    corrsavg.append([])
    for j in range(Nbins):
        corrsavg[i].append(np.sum(corrs[-1][j*Ni:(j+1)*Ni], 0) / Ni)

# mean correlations
corrsavg = np.array(corrsavg) # shape: (Ls, Nbins, 3/6 corr types)
corravg_horhor = np.sum(corrsavg[:, :, 0], 1) / Nbins # shape: (Ls, L-1 floats)
corravg_verver = np.sum(corrsavg[:, :, 1], 1) / Nbins
corravg_verhor = np.sum(corrsavg[:, :, 2], 1) / Nbins
corravg_horver = np.sum(corrsavg[:, :, 3], 1) / Nbins
corravg_diag1  = np.sum(corrsavg[:, :, 4], 1) / Nbins # shape: (Ls, 2L-1 floats)
corravg_diag2  = np.sum(corrsavg[:, :, 5], 1) / Nbins

corravg_along = np.sum(corrsavg[:, :, 0] +
                       corrsavg[:, :, 1], 1) / (2 * Nbins) # shape: (Ls, L-1 floats)
corravg_prep  = np.sum(corrsavg[:, :, 2] +
                       corrsavg[:, :, 3], 1) / (2 * Nbins)
corravg_diag  = np.sum(corrsavg[:, :, 4] +
                       corrsavg[:, :, 5], 1) / (2 * Nbins) # shape: (Ls, 2L-1 floats)

# standard deviations. maybe should divide by sqrt(Ni) instead?
erravg_horhor = np.array([np.std(corrsavg[i, :, 0]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_verver = np.array([np.std(corrsavg[i, :, 1]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_verhor = np.array([np.std(corrsavg[i, :, 2]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_horver = np.array([np.std(corrsavg[i, :, 3]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag1  = np.array([np.std(corrsavg[i, :, 4]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag2  = np.array([np.std(corrsavg[i, :, 5]) for i in range(len(Ls))]) / np.sqrt(Niter)

erravg_along = np.array([np.std((corrsavg[i, :, 0] +
                                 corrsavg[i, :, 1]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_prep  = np.array([np.std((corrsavg[i, :, 2] +
                                 corrsavg[i, :, 3]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag  = np.array([np.std((corrsavg[i, :, 4] +
                                 corrsavg[i, :, 5]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)

# save corrs & errs to files
for i in range(len(Ls)):
    np.savez('data/L%s_N%s_%sx%sbins_all.npz' % (Ls[i], Niter, Ni, Nbins),
             corrs=[corravg_horhor[0], corravg_verver[0], corravg_verhor[0],
                    corravg_horver[0], corravg_diag1[0], corravg_diag2[0]],
             errs=[erravg_horhor[0], erravg_verver[0], erravg_verhor[0],
                   erravg_horver[0], erravg_diag1[0], erravg_diag2[0]])
    
    np.savez('data/L%s_N%s_%sx%sbins_final.npz' % (Ls[i], Niter, Ni, Nbins),
             corrs=[corravg_along[0], corravg_prep[0], corravg_diag[0]],
             errs=[erravg_along[0], erravg_prep[0], erravg_diag[0]])

In [346]:
# Testing:
''' corrs             : list  of Ls       lists
    corrs[0]          : list  of Niter    tuples
    corrs[0][0]       : tuple of 3/6      arrays
    corrs[0][0][0]    : array of L-1/2L-1 floats
    corrs[0][0][0][0] : float
'''
''' corrsavg             : list  of Ls       lists
    corrsavg[0]          : list  of Nbins    arrays
    corrsavg[0][0]       : array of 3/6      arrays
    corrsavg[0][0][0]    : array of L-1/2L-1 floats
    corrsavg[0][0][0][0] : float
'''
np.shape(corrs), corrsavg.shape, corravg_horhor[0].shape, erravg_horhor[0].shape

((1, 100, 6), (1, 10, 6), (9,), (9,))

In [8]:
Ls = [20] # [10, 20, 30, 40, 50]
Ni = 100000 / Ls[0]**2
Nbins = 20
Niter = Ni * Nbins
corrs = [] # shape: (Ls, Niter, 3/6 corr types)
corrsavg = [] # shape: (Ls, Nbins, 3/6 corr types)

# run worm algorithm & get correlations
for i, L in enumerate(Ls):
    corrs.append(run_worm_simulation(L, L, Niter, corr_type='all'))
    
    # Binning analysis
    corrsavg.append([])
    for j in range(Nbins):
        corrsavg[i].append(np.sum(corrs[-1][j*Ni:(j+1)*Ni], 0) / Ni)

# mean correlations
corrsavg = np.array(corrsavg) # shape: (Ls, Nbins, 3/6 corr types)
corravg_horhor = np.sum(corrsavg[:, :, 0], 1) / Nbins # shape: (Ls, L-1 floats)
corravg_verver = np.sum(corrsavg[:, :, 1], 1) / Nbins
corravg_verhor = np.sum(corrsavg[:, :, 2], 1) / Nbins
corravg_horver = np.sum(corrsavg[:, :, 3], 1) / Nbins
corravg_diag1  = np.sum(corrsavg[:, :, 4], 1) / Nbins # shape: (Ls, 2L-1 floats)
corravg_diag2  = np.sum(corrsavg[:, :, 5], 1) / Nbins

corravg_along = np.sum(corrsavg[:, :, 0] +
                       corrsavg[:, :, 1], 1) / (2 * Nbins) # shape: (Ls, L-1 floats)
corravg_prep  = np.sum(corrsavg[:, :, 2] +
                       corrsavg[:, :, 3], 1) / (2 * Nbins)
corravg_diag  = np.sum(corrsavg[:, :, 4] +
                       corrsavg[:, :, 5], 1) / (2 * Nbins) # shape: (Ls, 2L-1 floats)

# standard deviations. maybe should divide by sqrt(Ni) instead?
erravg_horhor = np.array([np.std(corrsavg[i, :, 0]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_verver = np.array([np.std(corrsavg[i, :, 1]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_verhor = np.array([np.std(corrsavg[i, :, 2]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_horver = np.array([np.std(corrsavg[i, :, 3]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag1  = np.array([np.std(corrsavg[i, :, 4]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag2  = np.array([np.std(corrsavg[i, :, 5]) for i in range(len(Ls))]) / np.sqrt(Niter)

erravg_along = np.array([np.std((corrsavg[i, :, 0] +
                                 corrsavg[i, :, 1]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_prep  = np.array([np.std((corrsavg[i, :, 2] +
                                 corrsavg[i, :, 3]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag  = np.array([np.std((corrsavg[i, :, 4] +
                                 corrsavg[i, :, 5]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)

# save corrs & errs to files
for i in range(len(Ls)):
    np.savez('data/L%s_N%s_%sx%sbins_all.npz' % (Ls[i], Niter, Ni, Nbins),
             corrs=[corravg_horhor[0], corravg_verver[0], corravg_verhor[0],
                    corravg_horver[0], corravg_diag1[0], corravg_diag2[0]],
             errs=[erravg_horhor[0], erravg_verver[0], erravg_verhor[0],
                   erravg_horver[0], erravg_diag1[0], erravg_diag2[0]])
    
    np.savez('data/L%s_N%s_%sx%sbins_final.npz' % (Ls[i], Niter, Ni, Nbins),
             corrs=[corravg_along[0], corravg_prep[0], corravg_diag[0]],
             errs=[erravg_along[0], erravg_prep[0], erravg_diag[0]])

In [9]:
Ls = [30] # [10, 20, 30, 40, 50]
Ni = 100000 / Ls[0]**2
Nbins = 20
Niter = Ni * Nbins
corrs = [] # shape: (Ls, Niter, 3/6 corr types)
corrsavg = [] # shape: (Ls, Nbins, 3/6 corr types)

# run worm algorithm & get correlations
for i, L in enumerate(Ls):
    corrs.append(run_worm_simulation(L, L, Niter, corr_type='all'))
    
    # Binning analysis
    corrsavg.append([])
    for j in range(Nbins):
        corrsavg[i].append(np.sum(corrs[-1][j*Ni:(j+1)*Ni], 0) / Ni)

# mean correlations
corrsavg = np.array(corrsavg) # shape: (Ls, Nbins, 3/6 corr types)
corravg_horhor = np.sum(corrsavg[:, :, 0], 1) / Nbins # shape: (Ls, L-1 floats)
corravg_verver = np.sum(corrsavg[:, :, 1], 1) / Nbins
corravg_verhor = np.sum(corrsavg[:, :, 2], 1) / Nbins
corravg_horver = np.sum(corrsavg[:, :, 3], 1) / Nbins
corravg_diag1  = np.sum(corrsavg[:, :, 4], 1) / Nbins # shape: (Ls, 2L-1 floats)
corravg_diag2  = np.sum(corrsavg[:, :, 5], 1) / Nbins

corravg_along = np.sum(corrsavg[:, :, 0] +
                       corrsavg[:, :, 1], 1) / (2 * Nbins) # shape: (Ls, L-1 floats)
corravg_prep  = np.sum(corrsavg[:, :, 2] +
                       corrsavg[:, :, 3], 1) / (2 * Nbins)
corravg_diag  = np.sum(corrsavg[:, :, 4] +
                       corrsavg[:, :, 5], 1) / (2 * Nbins) # shape: (Ls, 2L-1 floats)

# standard deviations. maybe should divide by sqrt(Ni) instead?
erravg_horhor = np.array([np.std(corrsavg[i, :, 0]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_verver = np.array([np.std(corrsavg[i, :, 1]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_verhor = np.array([np.std(corrsavg[i, :, 2]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_horver = np.array([np.std(corrsavg[i, :, 3]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag1  = np.array([np.std(corrsavg[i, :, 4]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag2  = np.array([np.std(corrsavg[i, :, 5]) for i in range(len(Ls))]) / np.sqrt(Niter)

erravg_along = np.array([np.std((corrsavg[i, :, 0] +
                                 corrsavg[i, :, 1]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_prep  = np.array([np.std((corrsavg[i, :, 2] +
                                 corrsavg[i, :, 3]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag  = np.array([np.std((corrsavg[i, :, 4] +
                                 corrsavg[i, :, 5]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)

# save corrs & errs to files
for i in range(len(Ls)):
    np.savez('data/L%s_N%s_%sx%sbins_all.npz' % (Ls[i], Niter, Ni, Nbins),
             corrs=[corravg_horhor[0], corravg_verver[0], corravg_verhor[0],
                    corravg_horver[0], corravg_diag1[0], corravg_diag2[0]],
             errs=[erravg_horhor[0], erravg_verver[0], erravg_verhor[0],
                   erravg_horver[0], erravg_diag1[0], erravg_diag2[0]])
    
    np.savez('data/L%s_N%s_%sx%sbins_final.npz' % (Ls[i], Niter, Ni, Nbins),
             corrs=[corravg_along[0], corravg_prep[0], corravg_diag[0]],
             errs=[erravg_along[0], erravg_prep[0], erravg_diag[0]])

In [10]:
Ls = [40] # [10, 20, 30, 40, 50]
Ni = 100000 / Ls[0]**2
Nbins = 10 
Niter = Ni * Nbins
corrs = [] # shape: (Ls, Niter, 3/6 corr types)
corrsavg = [] # shape: (Ls, Nbins, 3/6 corr types)

# run worm algorithm & get correlations
for i, L in enumerate(Ls):
    corrs.append(run_worm_simulation(L, L, Niter, corr_type='all'))
    
    # Binning analysis
    corrsavg.append([])
    for j in range(Nbins):
        corrsavg[i].append(np.sum(corrs[-1][j*Ni:(j+1)*Ni], 0) / Ni)

# mean correlations
corrsavg = np.array(corrsavg) # shape: (Ls, Nbins, 3/6 corr types)
corravg_horhor = np.sum(corrsavg[:, :, 0], 1) / Nbins # shape: (Ls, L-1 floats)
corravg_verver = np.sum(corrsavg[:, :, 1], 1) / Nbins
corravg_verhor = np.sum(corrsavg[:, :, 2], 1) / Nbins
corravg_horver = np.sum(corrsavg[:, :, 3], 1) / Nbins
corravg_diag1  = np.sum(corrsavg[:, :, 4], 1) / Nbins # shape: (Ls, 2L-1 floats)
corravg_diag2  = np.sum(corrsavg[:, :, 5], 1) / Nbins

corravg_along = np.sum(corrsavg[:, :, 0] +
                       corrsavg[:, :, 1], 1) / (2 * Nbins) # shape: (Ls, L-1 floats)
corravg_prep  = np.sum(corrsavg[:, :, 2] +
                       corrsavg[:, :, 3], 1) / (2 * Nbins)
corravg_diag  = np.sum(corrsavg[:, :, 4] +
                       corrsavg[:, :, 5], 1) / (2 * Nbins) # shape: (Ls, 2L-1 floats)

# standard deviations. maybe should divide by sqrt(Ni) instead?
erravg_horhor = np.array([np.std(corrsavg[i, :, 0]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_verver = np.array([np.std(corrsavg[i, :, 1]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_verhor = np.array([np.std(corrsavg[i, :, 2]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_horver = np.array([np.std(corrsavg[i, :, 3]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag1  = np.array([np.std(corrsavg[i, :, 4]) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag2  = np.array([np.std(corrsavg[i, :, 5]) for i in range(len(Ls))]) / np.sqrt(Niter)

erravg_along = np.array([np.std((corrsavg[i, :, 0] +
                                 corrsavg[i, :, 1]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_prep  = np.array([np.std((corrsavg[i, :, 2] +
                                 corrsavg[i, :, 3]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)
erravg_diag  = np.array([np.std((corrsavg[i, :, 4] +
                                 corrsavg[i, :, 5]) / 2) for i in range(len(Ls))]) / np.sqrt(Niter)

# save corrs & errs to files
for i in range(len(Ls)):
    np.savez('data/L%s_N%s_%sx%sbins_all.npz' % (Ls[i], Niter, Ni, Nbins),
             corrs=[corravg_horhor[0], corravg_verver[0], corravg_verhor[0],
                    corravg_horver[0], corravg_diag1[0], corravg_diag2[0]],
             errs=[erravg_horhor[0], erravg_verver[0], erravg_verhor[0],
                   erravg_horver[0], erravg_diag1[0], erravg_diag2[0]])
    
    np.savez('data/L%s_N%s_%sx%sbins_final.npz' % (Ls[i], Niter, Ni, Nbins),
             corrs=[corravg_along[0], corravg_prep[0], corravg_diag[0]],
             errs=[erravg_along[0], erravg_prep[0], erravg_diag[0]])

In [ ]:
# Plots
Ls = [10, 20, 30, 40]
Ni = 1000
Nbins = 20
Niter = Ni * Nbins

for L in Ls:
    data_all = np.load('data/L%s_N%s_%sx%sbins_all.npz' % (L, Niter, Ni, Nbins))
    data_final = np.load('data/L%s_N%s_%sx%sbins_final.npz' % (L, Niter, Ni, Nbins))
    corrs, errs = data['corrs'], data['errs']
    
    plt.errorbar(corrs[0], errs[0], 'o', label='L = %s, #iter: %s' % (L, Niter))

# ...

plt.show()